## COMBS Tutorial
### Import COMBS and related packages

In [1]:
import combs2
import prody as pr
import pandas as pd
import pickle
pd.set_option("display.max_columns", None)


### Setup template PDBs as ProDy AtomGroup objects

In [2]:
input_dir = '/Users/npolizzi/Projects/design/Combs2/tutorials/files/jupyter_notebook/'

In [3]:
gly_pdb_name = '00009.f63440efff7e.allbb_ala_min_gly_0001.pdb'
ala_pdb_name = '00009.f63440efff7e.allbb_ala_min_ala_0001.pdb'
pdb_gly_path = input_dir + gly_pdb_name
pdb_ala_path = input_dir + ala_pdb_name

pdb_gly = pr.parsePDB(pdb_gly_path)
pdb_ala = pr.parsePDB(pdb_ala_path)

@> 1039 atoms and 1 coordinate set(s) were parsed in 0.02s.
@> 1483 atoms and 1 coordinate set(s) were parsed in 0.02s.


### Use ProDy PDB objects to setup a COMBS template

In [4]:
template = combs2.design.template.Template(pdb_gly)

Define the surface of the protein via its alpha hull, here with radius = 9 Ang.

In [5]:
template.set_alpha_hull(pdb_ala, alpha=9)

### Write a residue file for design with COMBS.

In [6]:
resnums = [21, 28, 44, 48, 51, 55, 95, 99, 102]
chains = ['A'] * len(resnums)
segs = ['A'] * len(resnums)
segs_chains_resnums = zip(segs, chains, resnums)

outpath_resfile = input_dir

combs2.design.functions.write_resfile(template,
                                      CGs=['bb_cco', 'conh2'],
                                      outpath=outpath_resfile,
                                      filename='resfile',
                                      tag='',
                                      resindices=None,
                                      segs_chains_resnums=segs_chains_resnums,
                                      pikaa_dict=None,
                                      bb_dep=1,
                                      use_enriched_vdMs=True,
                                      CA_burial_distance=None,
                                      exclude_exposed=False,
                                      exclude_intermed=False,
                                      exclude_buried=False,
                                      top_exposed=2,
                                      top_intermed=None,
                                      top_buried=None,
                                      alpha_hull_radius=9,
                                      use_propensities=True,
                                      propensity_threshold=0.9,
                                      use_abple=True,
                                      use_dssp=False,
                                      path_to_pdb_for_dssp=None,
                                      allowed_exposed='KRDENQSTMAGP',
                                      allowed_intermed='NQSTCMAGPVIL',
                                      allowed_buried='AGSTMCPVILHFWY',
                                      hb_only_residues='',
                                      all_contact_residues='GHFWY')

Peak into the resfile:

In [7]:
with open(input_dir + 'resfile.txt', 'r') as infile:
    for _ in range(15):
        print(infile.readline().strip())

48 A A, PIKAA PQRNTM, CG bb_cco, bbdep 1, top 2, enriched, hbond_only
48 A A, PIKAA G, CG bb_cco, bbdep 1, top 2, enriched
48 A A, PIKAA PSDQRNTM, CG conh2, bbdep 1, top 2, enriched, hbond_only
55 A A, PIKAA PQRNTM, CG bb_cco, bbdep 1, top 2, enriched, hbond_only
55 A A, PIKAA G, CG bb_cco, bbdep 1, top 2, enriched
55 A A, PIKAA PSDQRNTM, CG conh2, bbdep 1, top 2, enriched, hbond_only
99 A A, PIKAA PQRNTM, CG bb_cco, bbdep 1, top 2, enriched, hbond_only
99 A A, PIKAA G, CG bb_cco, bbdep 1, top 2, enriched
99 A A, PIKAA PSDQRNTM, CG conh2, bbdep 1, top 2, enriched, hbond_only
21 A A, PIKAA PVLCTIM, CG bb_cco, bbdep 1, enriched, hbond_only
21 A A, PIKAA FWHGY, CG bb_cco, bbdep 1, enriched
21 A A, PIKAA TMPS, CG conh2, bbdep 1, enriched, hbond_only
21 A A, PIKAA WHYF, CG conh2, bbdep 1, enriched
28 A A, PIKAA PVLCTIM, CG bb_cco, bbdep 1, enriched, hbond_only
28 A A, PIKAA FWHGY, CG bb_cco, bbdep 1, enriched


### Make a COMBS Sample object to sample van der Mers (vdMs).

In [8]:
#path to COMBS residue file for design
path_to_resfile= input_dir + 'resfile.txt'

# path to vdM databases
path_to_database='/Volumes/disk1/Combs2/database/20211005/vdMs/'

s = combs2.design._sample.Sample(**dict(path_to_resfile=path_to_resfile,
                                        path_to_database=path_to_database))

# Read the resfile into the Sample object
s.read_resfile()

### An aside about vdM databases...
The vdM databases are Pandas dataframes; they contain a lot of information.  Below, the first 50 rows of the dataframe show two "instances" of SER / CONH2 vdMs.  Each vdM has a unique identifier comprised of 3 keys: *CG*, *rota*, and *probe_name*.  The chemical group (CG) of the vdM is always **Y** in column *chain* with *resnum*=10, and the contacting amino-acid is **X** in *chain* with *resnum*=10.  The dataframes also contain the backbone coordinates and other info for the flanking residues (i+-1) of chain X (resnums 9 and 11, respectively).

In [9]:
df = pd.read_parquet(path_to_database + 'conh2/SER.parquet.gzip')
df.head(50)

,resnum,chain,resname,name,beta,occ,c_x,c_y,c_z,c_D_x,c_D_y,c_D_z,c_H1_x,c_H1_y,c_H1_z,c_H2_x,c_H2_y,c_H2_z,c_H3_x,c_H3_y,c_H3_z,c_H4_x,c_H4_y,c_H4_z,c_A1_x,c_A1_y,c_A1_z,c_A2_x,c_A2_y,c_A2_z,atom_type_label,chi1,chi2,chi3,chi4,evaluation,rotamer,rscc,rsr,rsrz,phi,psi,rama,pdb_chain,pdb_segment,pdb_resnum,dssp,dssp_acc,dssp_seq,ABPLE_seq,ABPLE,ABPLE_3mer,dssp_3mer,contact_hb,contact_wh,contact_cc,contact_so,partners_hb,partners_wh,partners_cc,partners_so,rota,CG,seq,hull_status,dist_to_hull,pdb_name,score_index,probe_name,avg_bb_beta,sigma_bb_beta,resname_rota,centroid,cluster_number,cluster_size,C_score_bb_ind,maxdist_to_centroid,cluster_atom,cluster_order,centroid_ABPLE_E,centroid_ABPLE_L,centroid_ABPLE_A,centroid_ABPLE_P,centroid_ABPLE_B,centroid_dssp_E,centroid_dssp_I,centroid_dssp_T,centroid_dssp_C,centroid_dssp_S,centroid_dssp_H,centroid_dssp_G,centroid_dssp_B,centroid_hb_bb_ind,C_score_ABPLE_A,C_score_ABPLE_B,C_score_ABPLE_E,C_score_ABPLE_L,C_score_ABPLE_P,C_score_dssp_B,C_score_dssp_C,C_score_dssp_E,C_score_dssp_G,C_score_dssp_H,C_score_dssp_I,C_score_dssp_S,C_score_dssp_T,C_score_hb_ABPLE_A,C_score_hb_ABPLE_B,C_score_hb_ABPLE_E,C_score_hb_ABPLE_L,C_score_hb_ABPLE_P,C_score_hb_bb_ind,C_score_hb_dssp_B,C_score_hb_dssp_C,C_score_hb_dssp_E,C_score_hb_dssp_G,C_score_hb_dssp_H,C_score_hb_dssp_I,C_score_hb_dssp_S,C_score_hb_dssp_T,cluster_rank_ABPLE_A,cluster_rank_ABPLE_B,cluster_rank_ABPLE_E,cluster_rank_ABPLE_L,cluster_rank_ABPLE_P,cluster_rank_dssp_B,cluster_rank_dssp_C,cluster_rank_dssp_E,cluster_rank_dssp_G,cluster_rank_dssp_H,cluster_rank_dssp_I,cluster_rank_dssp_S,cluster_rank_dssp_T,cluster_rank_hb_ABPLE_A,cluster_rank_hb_ABPLE_B,cluster_rank_hb_ABPLE_E,cluster_rank_hb_ABPLE_L,cluster_rank_hb_ABPLE_P,cluster_rank_hb_bb_ind,cluster_rank_hb_dssp_B,cluster_rank_hb_dssp_C,cluster_rank_hb_dssp_E,cluster_rank_hb_dssp_G,cluster_rank_hb_dssp_H,cluster_rank_hb_dssp_I,cluster_rank_hb_dssp_S,cluster_rank_hb_dssp_T,CG_rep_centroid_coarse_ABPLE,CG_rep_cluster_number_coarse_ABPLE,CG_rep_cluster_size_coarse_ABPLE,CG_rep_cluster_score_coarse_ABPLE,CG_rep_centroid_medium_ABPLE,CG_rep_cluster_number_medium_ABPLE,CG_rep_cluster_size_medium_ABPLE,CG_rep_cluster_score_medium_ABPLE,CG_rep_medium_ABPLE,CG_rep_centroid_fine_ABPLE,CG_rep_cluster_number_fine_ABPLE,CG_rep_cluster_size_fine_ABPLE,CG_rep_cluster_score_fine_ABPLE,CG_rep_fine_ABPLE,sc_rep_centroid_fine_ABPLE,sc_rep_cluster_number_fine_ABPLE,sc_rep_cluster_size_fine_ABPLE,sc_rep_cluster_score_fine_ABPLE,sc_rep_fine_ABPLE,sc_rep_centroid_fine_dssp,CG_rep_cluster_size_coarse_dssp,CG_rep_cluster_score_fine_dssp,sc_rep_fine_dssp,CG_rep_fine_dssp,sc_rep_cluster_score_fine_dssp,CG_rep_cluster_score_medium_dssp,sc_rep_cluster_size_fine_dssp,CG_rep_centroid_coarse_dssp,CG_rep_cluster_number_fine_dssp,CG_rep_cluster_number_coarse_dssp,CG_rep_cluster_size_fine_dssp,sc_rep_cluster_number_fine_dssp,CG_rep_cluster_number_medium_dssp,CG_rep_cluster_size_medium_dssp,CG_rep_cluster_score_coarse_dssp,CG_rep_centroid_medium_dssp,CG_rep_centroid_fine_dssp,CG_rep_medium_dssp,CG_rep_cluster_score_fine_bb_ind,CG_rep_cluster_size_fine_bb_ind,sc_rep_cluster_number_fine_bb_ind,CG_rep_cluster_score_coarse_bb_ind,sc_rep_centroid_fine_bb_ind,CG_rep_cluster_size_medium_bb_ind,CG_rep_centroid_coarse_bb_ind,CG_rep_cluster_size_coarse_bb_ind,sc_rep_cluster_size_fine_bb_ind,CG_rep_cluster_number_fine_bb_ind,CG_rep_centroid_fine_bb_ind,CG_rep_cluster_number_medium_bb_ind,CG_rep_cluster_score_medium_bb_ind,CG_rep_centroid_medium_bb_ind,sc_rep_fine_bb_ind,CG_rep_medium_bb_ind,CG_rep_cluster_number_coarse_bb_ind,CG_rep_fine_bb_ind,sc_rep_cluster_score_fine_bb_ind,contact_type
0,10,Y,GLN,CG,30.020000,1.0,-0.751365,-1.069795,-5.110119e+00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,c_alkyl,295.600006,181.399994,346.500000,NaN,Favored,mt0,0.951,0.091,-0.957,-68.199997,-36.599998,Favored,A,A,60,T,49.0,EEECSCTTTCSHH,BBPPBBAAABPAA,A,BAA,CTT,None,None,True,None,None,None,OG,None,1,1,LV

Here are only the first few rows of the dataframe, which makes it easier to scroll through all the columns.

In [10]:
df.head(3)

,resnum,chain,resname,name,beta,occ,c_x,c_y,c_z,c_D_x,c_D_y,c_D_z,c_H1_x,c_H1_y,c_H1_z,c_H2_x,c_H2_y,c_H2_z,c_H3_x,c_H3_y,c_H3_z,c_H4_x,c_H4_y,c_H4_z,c_A1_x,c_A1_y,c_A1_z,c_A2_x,c_A2_y,c_A2_z,atom_type_label,chi1,chi2,chi3,chi4,evaluation,rotamer,rscc,rsr,rsrz,phi,psi,rama,pdb_chain,pdb_segment,pdb_resnum,dssp,dssp_acc,dssp_seq,ABPLE_seq,ABPLE,ABPLE_3mer,dssp_3mer,contact_hb,contact_wh,contact_cc,contact_so,partners_hb,partners_wh,partners_cc,partners_so,rota,CG,seq,hull_status,dist_to_hull,pdb_name,score_index,probe_name,avg_bb_beta,sigma_bb_beta,resname_rota,centroid,cluster_number,cluster_size,C_score_bb_ind,maxdist_to_centroid,cluster_atom,cluster_order,centroid_ABPLE_E,centroid_ABPLE_L,centroid_ABPLE_A,centroid_ABPLE_P,centroid_ABPLE_B,centroid_dssp_E,centroid_dssp_I,centroid_dssp_T,centroid_dssp_C,centroid_dssp_S,centroid_dssp_H,centroid_dssp_G,centroid_dssp_B,centroid_hb_bb_ind,C_score_ABPLE_A,C_score_ABPLE_B,C_score_ABPLE_E,C_score_ABPLE_L,C_score_ABPLE_P,C_score_dssp_B,C_score_dssp_C,C_score_dssp_E,C_score_dssp_G,C_score_dssp_H,C_score_dssp_I,C_score_dssp_S,C_score_dssp_T,C_score_hb_ABPLE_A,C_score_hb_ABPLE_B,C_score_hb_ABPLE_E,C_score_hb_ABPLE_L,C_score_hb_ABPLE_P,C_score_hb_bb_ind,C_score_hb_dssp_B,C_score_hb_dssp_C,C_score_hb_dssp_E,C_score_hb_dssp_G,C_score_hb_dssp_H,C_score_hb_dssp_I,C_score_hb_dssp_S,C_score_hb_dssp_T,cluster_rank_ABPLE_A,cluster_rank_ABPLE_B,cluster_rank_ABPLE_E,cluster_rank_ABPLE_L,cluster_rank_ABPLE_P,cluster_rank_dssp_B,cluster_rank_dssp_C,cluster_rank_dssp_E,cluster_rank_dssp_G,cluster_rank_dssp_H,cluster_rank_dssp_I,cluster_rank_dssp_S,cluster_rank_dssp_T,cluster_rank_hb_ABPLE_A,cluster_rank_hb_ABPLE_B,cluster_rank_hb_ABPLE_E,cluster_rank_hb_ABPLE_L,cluster_rank_hb_ABPLE_P,cluster_rank_hb_bb_ind,cluster_rank_hb_dssp_B,cluster_rank_hb_dssp_C,cluster_rank_hb_dssp_E,cluster_rank_hb_dssp_G,cluster_rank_hb_dssp_H,cluster_rank_hb_dssp_I,cluster_rank_hb_dssp_S,cluster_rank_hb_dssp_T,CG_rep_centroid_coarse_ABPLE,CG_rep_cluster_number_coarse_ABPLE,CG_rep_cluster_size_coarse_ABPLE,CG_rep_cluster_score_coarse_ABPLE,CG_rep_centroid_medium_ABPLE,CG_rep_cluster_number_medium_ABPLE,CG_rep_cluster_size_medium_ABPLE,CG_rep_cluster_score_medium_ABPLE,CG_rep_medium_ABPLE,CG_rep_centroid_fine_ABPLE,CG_rep_cluster_number_fine_ABPLE,CG_rep_cluster_size_fine_ABPLE,CG_rep_cluster_score_fine_ABPLE,CG_rep_fine_ABPLE,sc_rep_centroid_fine_ABPLE,sc_rep_cluster_number_fine_ABPLE,sc_rep_cluster_size_fine_ABPLE,sc_rep_cluster_score_fine_ABPLE,sc_rep_fine_ABPLE,sc_rep_centroid_fine_dssp,CG_rep_cluster_size_coarse_dssp,CG_rep_cluster_score_fine_dssp,sc_rep_fine_dssp,CG_rep_fine_dssp,sc_rep_cluster_score_fine_dssp,CG_rep_cluster_score_medium_dssp,sc_rep_cluster_size_fine_dssp,CG_rep_centroid_coarse_dssp,CG_rep_cluster_number_fine_dssp,CG_rep_cluster_number_coarse_dssp,CG_rep_cluster_size_fine_dssp,sc_rep_cluster_number_fine_dssp,CG_rep_cluster_number_medium_dssp,CG_rep_cluster_size_medium_dssp,CG_rep_cluster_score_coarse_dssp,CG_rep_centroid_medium_dssp,CG_rep_centroid_fine_dssp,CG_rep_medium_dssp,CG_rep_cluster_score_fine_bb_ind,CG_rep_cluster_size_fine_bb_ind,sc_rep_cluster_number_fine_bb_ind,CG_rep_cluster_score_coarse_bb_ind,sc_rep_centroid_fine_bb_ind,CG_rep_cluster_size_medium_bb_ind,CG_rep_centroid_coarse_bb_ind,CG_rep_cluster_size_coarse_bb_ind,sc_rep_cluster_size_fine_bb_ind,CG_rep_cluster_number_fine_bb_ind,CG_rep_centroid_fine_bb_ind,CG_rep_cluster_number_medium_bb_ind,CG_rep_cluster_score_medium_bb_ind,CG_rep_centroid_medium_bb_ind,sc_rep_fine_bb_ind,CG_rep_medium_bb_ind,CG_rep_cluster_number_coarse_bb_ind,CG_rep_fine_bb_ind,sc_rep_cluster_score_fine_bb_ind,contact_type
0,10,Y,GLN,CG,30.020000,1.0,-0.751365,-1.069795,-5.110119,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,c_alkyl,295.600006,181.399994,346.5,NaN,Favored,mt0,0.951,0.091,-0.957,-68.199997,-36.599998,Favored,A,A,60,T,49.0,EEECSCTTTCSHH,BBPPBBAAABPAA,A,BAA,CTT,None,None,True,None,None,None,OG,None,1,1,LVSAIMQSVSG

***Note:*** The **pdb_resnum** and **pdb_chain** columns of the dataframes refer to the raw residue number and chain in the pdb library that was searched.  However, some of the PDB files had their chain names changed to unique letters, so the chain identifier may not correspond to the same one in the PDB file (if you were to re-download it from the RCSB, for example).

vdM instances can be printed as a PDB file by first selecting a unique instance from its identifier tags.  For example, the first vdM instance is:

In [12]:
df[['CG', 'rota', 'probe_name']].iloc[0]

CG                            1
rota                          1
probe_name    1AK5_biomol_1_A_A
Name: 0, dtype: object

To print this vdM instance:

In [13]:
v = df[(df['CG'] == 1) & (df['rota'] == 1) & (df['probe_name'] == '5KRD_biomol_2_A_A')]
vdM_output_dir = input_dir + 'vdM/'
combs2.design.functions.print_dataframe(v, outpath=vdM_output_dir, tag='_ser_conh2')

vdM clusters can also be printed as PDB files:

In [14]:
clu_output_dir = input_dir + 'vdM_cluster_1/'
combs2.design.functions.print_cluster(df, cluster_num=1, max_members=30, outpath=clu_output_dir, tag='_ser_conh2')

### Load vdMs onto the template backbone, removing clashing vdMs.

In [15]:
s.load_vdms(template, filter_by_phi_psi=False, run_parallel=False)

Loading bb_cco
Loaded  36301 vdMs of bb_cco
Loading conh2
Loaded  17611 vdMs of conh2


### Set ligand atoms <---> CG atoms correspondence for lookup in NearestNeighbors objects of CGs, which are created in next step.

In [16]:
path_to_ligand_file = input_dir + 'ligand.txt'
s.set_ligand_vdm_correspondence(path_to_ligand_file)

### Create and fit NearestNeighbors objects for each CG type loaded.
Neighbors are defined by a distance metric, here RMSD in Angstroms, which is tunable.

In [17]:
s.set_cg_neighbors(max_dist_criterion=True, cg_max_dists=dict(conh2=0.8, bb_cco=0.8))

Setting neighbors...


### Load vdM-superimposed ligands onto the template backbone, removing clashing ligands and those ligands that do not pass a burial filter.

In [19]:
path_to_ligand_database='/Volumes/disk1/Combs2/database/20211005/lig_vdMs/apixaban/GG2/enriched/'

In [ ]:
# #Uncomment this cell to load ligands from the precomputed database
# s.load_ligands(template, use_ligs_of_loaded_vdms_only=True,
#                     frac_non_hb_heavy_buried=0.5,
#                     path_to_ligand_database=path_to_ligand_database,
#                     hull_tolerance=0,
#                     run_parallel=False)

If loading ligands in parallel is desirable, it can currently be done the following way.

In [20]:
# #Uncomment this cell to load ligands in parallel from the precomputed database
# s_ligs = s.__copy__(no_vdms=True)
# s_ligs.load_ligands(template, use_ligs_of_loaded_vdms_only=True,
#                     frac_non_hb_heavy_buried=0.5,
#                     path_to_ligand_database=path_to_ligand_database,
#                     hull_tolerance=0,
#                     run_parallel=True)
# s.set_loaded_ligand_data(s_ligs)

Loading ligands of bb_cco
    cg_group 2
    cg_group 3
    cg_group 4
    cg_group 1
    cg_group 5
Loading ligands of conh2
    cg_group 1
Loaded  142 ligands


### Load ligands by superimposing on-the-fly

Pre-computed ligand libraries can speed up the loading process but do not need to be made.  Instead, one ligand conformer can be loaded, and this conformer can be superimposed onto CGs of loaded vdMs.  Here's how.

In [18]:
lig_dir = '/Users/npolizzi/Projects/design/Combs2/tutorials/files/HPC_scripts/superpose_ligand/'

s.load_ligand_conformer(path_to_lig_pdb=lig_dir + 'GG2.pdb', 
                        path_to_lig_params=lig_dir + 'GG2.params',
                        lig_resname='GG2', 
                        remove_atom_from_hb_dict=['N2', 'N5'], 
                        ligand_dataframe=None)

@> 59 atoms and 1 coordinate set(s) were parsed in 0.00s.


Ligand atom types and H-bonding atoms are automatically generated from the params file.  Here, two atoms (N2 and N5) that get marked by the code as H-bonding in the ligand (and are not really H-bonding) are removed from the H-bonding dictionary that tells Combs which atoms can H-bond. 

The ligand conformer can also be loaded as a pandas dataframe with the *ligand_dataframe* option, where the atom types and H-bonding atoms can be set manually, if desired.  

Multiple conformers can be loaded together by listing multiple pdb paths and using *load_ligand_conformers*:

In [19]:
pdb_paths = [lig_dir + 'GG2.pdb', ] # add more pdb paths to this list if desired.

s.load_ligand_conformers(paths_to_lig_pdbs=pdb_paths, 
                         path_to_lig_params=lig_dir + 'GG2.params',
                         lig_resname='GG2', 
                         remove_atom_from_hb_dict=['N2', 'N5'], 
                         ligand_dataframes=None)

@> 59 atoms and 1 coordinate set(s) were parsed in 0.00s.


The loaded conformer gets stored in the *ligand_conformers* attribute of the **Sample** object:

In [20]:
s.ligand_conformers[0]

,resnum,chain,segment,resname,name,c_x,c_y,c_z,c_D_x,c_D_y,c_D_z,c_H1_x,c_H1_y,c_H1_z,c_H2_x,c_H2_y,c_H2_z,c_H3_x,c_H3_y,c_H3_z,c_H4_x,c_H4_y,c_H4_z,c_A1_x,c_A1_y,c_A1_z,c_A2_x,c_A2_y,c_A2_z,atom_type_label,seg_chain_resnum,lig_resname,lig_name
0,1,B,A,GG2,C1,28.888000,25.850000,14.054000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,c_aro,"(A, B, 1)",GG2,C1
1,1,B,A,GG2,C10,32.425999,23.558001,18.648001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,c_aro,"(A, B, 1)",GG2,C10
2,1,B,A,GG2,C11,33.776001,23.631001,19.343000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,co,"(A, B, 1)",GG2,C11
3,1,B,A,GG2,C12,31.372000,22.576000,18.958000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,c_aro,"(A, B, 1)",GG2,C12
4,1,B,A,GG2,C13,30.294001,22.912001,18.093000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,c_aro,"(A, B, 1)",GG2,C13
5,1,B,A,GG2,C14,25.664000,19.604000,18.638000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,c_aro,"(A, B, 1)",GG2,C14
6,1,B,A,GG2,C15,27.289000,25.870001,12.400000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,c_alkyl,"(A, B, 1)",GG2,C15
7,1,B,A,GG2,C16,25.964001,18.275000,18.173000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,c_aro,"(A, B, 1)",GG2,C16
8,1,B,A,GG2,C17,31.273001,21.423000,19.955000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,c_alkyl,"(A, B, 1)",GG2,C17
9,1,B,A,GG2,C18,27.238001,17.816999,17.879999,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,c_aro,"(A, B, 1)",GG2,C18


The conformer can now be superimposed onto CGs of the loaded vdMs.

In [21]:
# Superpose in series (perhaps best for an HPC job)
s.superpose_ligands_to_CGs(template, frac_non_hb_heavy_buried=0.5, hull_tolerance=0, chunk_size=10000)

Superposing ligands of bb_cco 1 for conformer 1 ...
     10000 ligands entering clash/burial filters...
         35 ligands passed clash/burial filters.
     10000 ligands entering clash/burial filters...
         42 ligands passed clash/burial filters.
     10000 ligands entering clash/burial filters...
         13 ligands passed clash/burial filters.
     6301 ligands entering clash/burial filters...
         16 ligands passed clash/burial filters.
Superposing ligands of bb_cco 2 for conformer 1 ...
     10000 ligands entering clash/burial filters...
         3 ligands passed clash/burial filters.
     10000 ligands entering clash/burial filters...
         5 ligands passed clash/burial filters.
     10000 ligands entering clash/burial filters...
         17 ligands passed clash/burial filters.
     6301 ligands entering clash/burial filters...
         9 ligands passed clash/burial filters.
Superposing ligands of bb_cco 3 for conformer 1 ...
     10000 ligands entering clash/burial 

In [ ]:
# # Or superpose in parallel (will execute more quickly if you have the available cpus and memory)
# # Uncomment this cell to load ligands in parallel
# s_ligs = s.__copy__(no_vdms=True, keep_nbrs=True)
# s_ligs.superpose_ligands_to_CGs(template, frac_non_hb_heavy_buried=0.5, hull_tolerance=0,
#                                 chunk_size=10000, run_parallel=True)
# s.set_loaded_ligand_data(s_ligs)

### Find CG neighbors of corresponding atoms in each ligand.

In [22]:
s.find_ligand_cg_neighbors(maxdists=dict(conh2=0.8, bb_cco=0.8))

Finding neighbors of bb_cco 1 ...
Finding neighbors of bb_cco 2 ...
Finding neighbors of bb_cco 3 ...
Finding neighbors of bb_cco 4 ...
Finding neighbors of bb_cco 5 ...
Finding neighbors of conh2 1 ...


### Define constraints on the ligand/vdM poses for filtering.
Constraints such as atom burial, ligand/vdM atom contact type, etc, are possible.  See lig_csts.txt for examples.

In [23]:
path_to_constraint_file = input_dir + 'lig_csts.txt'
s.set_constraints(path_to_constraint_file)

### Find poses
With the neighbors and constraints now defined, proceed by finding the poses that pass the constraint filters.


In [24]:
s.find_poses(template)

A total of 8 poses were found.


If many poses are found, their subsequent analysis can be time-consuming. In this case, poses can be prioritized and only the "top" poses submitted for subsequent analysis.  

In [25]:
s.find_poses(template, filter_ligands_by_cluster=True, lig_rmsd_cutoff=1.0, 
             min_ligands_per_cluster=1, top_percent_per_cluster=0.1,
             only_top_percent=0.1, min_poses=50, max_poses=300)

Applying cluster filter to 396 ligands...
	  296 ligands remain...
Applying top-percent filter to 296 ligands...
	  50 ligands remain...
A total of 3 poses were found.


In the above, ligands will be clustered with a RMSD cutoff of 1.0 Angstroms and filtered by cluster (top 10 percent of ligands in each cluster will be used, or a minimum of 1 ligand per cluster, where "top" refers to designability, see below).  By setting *only_top_percent*=0.1, the top ten percent of remaining ligands will then be used, with a minimum of 50 poses (or less, if 10 percent of the ligands is less than 50), and a maximum of 300 ligands (if 10 percent is greater than 300).  Here, "designability" is a heuristic score, defined as the sum of the log of the number of vdM neighbors to each ligand fragment ("CG ligand coverage number" in ligand.txt).  It prioritizes ligands that have the largest number of vdM neighbors from dispersed sites.

Weights can be supplied to the heuristic ranking function at the level of the "CG ligand coverage number" (the last column in the ligand.txt file).  For example, the following will more heavily weight the vdMs that have CG ligand coverage number 2 (which corresponds to one of the carbonyls of the ligand, in this case) in the ligand.txt file, so that poses will be more highly ranked if they contain a vdM that covers this part of the ligand.

In [ ]:
# # Uncomment this cell to use weights for the heuristic ranking
# vals = [1, 10, 1] # There are 3 different CG ligand coverage numbers in the ligand.txt file.

# # make weights dictionary, where keys are CG ligand coverage number, values are the weights
# weights = {i: vals[i-1] for i in range(1, len(vals) + 1)}

# s.find_poses(template, filter_ligands_by_cluster=True, lig_rmsd_cutoff=1.0, 
#              min_ligands_per_cluster=1, top_percent_per_cluster=0.1,
#              only_top_percent=0.1, min_poses=50, max_poses=300, weight_dict=weights)

### Score poses
Now score the poses that passed the constraint filters.  If using only vdMs that are H-bonding, turn on use_hb_scores.  Otherwise, if using a mixture of non-H-bonding and H-bonding vdMs, it is more fair to score from the same distributions, so set use_hb_scores=False.

In [26]:
s.score_poses(template, bbdep=True, use_hb_scores=False,
              return_top_scoring_vdMs_only=False)

DEE + brute force
0 -1.7145644426345825 [(13, 3, '5TNX_biomol_1_A_A', 'bb_cco', ('A', 'A', 44))]
DEE + brute force
1 -2.7132129669189453 [(4, 1, '3H1N_biomol_1_A_A', 'bb_cco', ('A', 'A', 102))]
DEE + brute force
2 -3.8635698556900024 [(16, 1, '4O4V_biomol_1_A_A', 'bb_cco', ('A', 'A', 102)), (25, 1, '4XWH_biomol_1_A_A', 'bb_cco', ('A', 'A', 95))]


### Save to disk
The Sample object can be saved to disk.  This is convenient to avoid re-loading/pruning vdMs and ligands, but it can take up significant disk space (GBs, depending on how many vdMs/ligands were loaded).

In [23]:
# # Uncomment this cell to save the full sample object to disk.
# output_dir = input_dir + 'output/'
# s.save(outpath=output_dir, filename='sample_tutorial.pkl')

The Sample object can also be saved without the loaded vdMs or ligands, which saves significant disk space.  In this way, the poses can be conveniently saved for future computations.

In [27]:
output_dir = input_dir + 'output/'
s.save(outpath=output_dir, filename='sample_tutorial_poses.pkl', minimal_info_and_poses=True)

### Find buried, non-H-bonded, polar atoms
Get the top scoring poses and compute the number and identities of any buried, "unsatisfied", polar atoms in the vdMs or ligands of the optimum ligand/vdM set.

In [28]:
poses = s.get_top_poses(top=10)
s.set_buried_unsatisfied(poses, template)

### Print the top poses as PDB files.

In [29]:
output_dir = input_dir + 'output/'
for pose in poses:
    pose.print_opt_pdb(template, outdir=output_dir,
                       include_CG=True, label_vdM_segment_X=True)

Print the top pose without the CGs:

In [30]:
poses[0].print_opt_pdb(template, outdir=output_dir,
                       include_CG=False, label_vdM_segment_X=False, tag='_no_CG')

### Print score files of top poses.

In [31]:
for pose in poses:
    pose.print_to_energy_table(outdir=output_dir, filename_tag='_' + gly_pdb_name)

The top poses can be saved directly to disk by:

In [32]:
with open(output_dir + 'poses.pkl', 'wb') as outfile:
    pickle.dump(poses, outfile)

### Use stored poses as input for another Combs computation.

Saved poses can be loaded into a new Sample object by:

In [33]:
s_new = combs2.design._sample.Sample() # Create sample, resfile, and load vdMs the usual way

with open(output_dir + 'poses.pkl', 'rb') as infile:
    poses = pickle.load(infile)

s_new.load_poses(poses)

Ligands from these poses can now be used to lookup neighbors of CGs from vdMs loaded into s_new:

In [ ]:
# #Finding ligand neighbors
# s_new.find_ligand_cg_neighbors(dists=dict(pro=0.8))

# #Add neighboring vdMs to poses
# s_new.add_vdms_to_poses(use_optimum_vdms_only=True, freeze_optimum_vdms=True)

The above code assumes that vdMs of the "pro" CG were already loaded into the s_new object (not shown).  CG neighbors are computed to the ligands in the loaded poses (the CG-to-ligand atom correspondence file (ligand.txt) must exist and have already been loaded).  Next, the poses are updated with the new neighbors.  A new optimum choice of vdMs will be found using *s.score_poses*, and *use_optimum_vdms_only*=True will use only the previously found optimum vdMs in this new calculation (along with, of course, the newly added "pro" vdMs in this example). The option *freeze_optimum_vdms*=True will ensure the new optimum solution will include each vdM of the previously optimum solution.  So, if a "pro" vdM scored favorably, but were mutually inconsistent with a previously optimum vdM, the new pro vdM would not be substituted for the older vdM (even if its score were better).

### vdM Analysis:  Find vdMs in an input PDB file.

For a given input structure, the lookup module **run_lookup_ligand.py** will find the vdMs to a ligand:

```
> path_to_lookup=~/Projects/design/Combs2/combs2/programs/run_lookup_ligand.py
> path_to_lig_txt=../ligand.txt
> path_to_pdb=1_pose1_no_CG.pdb
> path_to_db=/Volumes/disk1/Combs/probe_Qbits_2p8/20211005/vdMs/
> path_to_lig_params=../../HPC_scripts/superpose_ligand/GG2.params
> outpath=./
> path_to_probe=~/Projects/design/Combs2/combs2/programs/probe
> conda activate env_combs #activate combs python environment
> python $path_to_lookup --lig=$path_to_lig_txt --pdb=$path_to_pdb --db=$path_to_db --o=$outpath --lig_params=$path_to_lig_params --probe=$path_to_probe
```

This will output a csv file that contains the closest-matched vdMs to protein-ligand interactions in the PDB file.

In [31]:
lig_vdms = pd.read_csv(input_dir + 'output/1_pose1_no_CG_GG2.csv')
lig_vdms.head(10)

,chain_rota,resnum_rota,resname_rota,chain_CG,resnum_CG,CG_type,CG_group,cluster_number,C_score_bb_ind,C_score_hb_bb_ind,C_score_ABPLE,C_score_hb_ABPLE,match_type,rmsd_to_centroid,max_dist_to_centroid
0,A,95,TYR,L,1,bb_cco,4,51,1.716003,0.279108,1.402268,0.019366,exact,0.366075,0.671119
1,A,95,TYR,L,1,bb_cco,3,14,2.546934,1.877964,2.713213,2.178850,exact,0.384667,0.618797
2,A,102,PHE,L,1,bb_cco,2,163,0.662221,NaN,1.150357,NaN,exact,0.267142,0.537662
3,A,102,PHE,L,1,bb_cco,1,284,0.151395,NaN,0.836699,NaN,exact,0.354119,0.579181


To look up vdMs of protein-protein interactions, **run_lookup.py** can be used:

```
> path_to_lookup=~/Projects/design/Combs2/combs2/programs/run_lookup.py
> path_to_pdb=6w70a.pdb
> path_to_db=/Volumes/disk1/Combs/probe_Qbits_2p8/20211005/vdMs/
> outpath=./
> path_to_probe=~/Projects/design/Combs2/combs2/programs/probe
> conda activate env_combs #activate combs python environment
> python $path_to_lookup --pdb=$path_to_pdb --db=$path_to_db --o=$outpath --probe=$path_to_probe
```

### Load rotamer-filtered vdMs, cluster the CGs, and print top clusters.

In [32]:
pdb_name = '6w70a.pdb'
pdb = pr.parsePDB(input_dir + pdb_name)
template = combs2.design.template.Template(pdb)
template.set_alpha_hull(pdb, alpha=9)

@> 1905 atoms and 1 coordinate set(s) were parsed in 0.02s.


In [33]:
path_to_phenix = '/Users/npolizzi/Applications/MolProbity/build/bin/phenix.rotalyze'
template.rotalyze(pdb_path=input_dir + pdb_name,
 path_to_phenix_rotalyze=path_to_phenix)

Use the *rotamers* option to load rotamer-specific vdMs.

In [34]:
outpath_resfile = input_dir
segs_chains_resnums = [('','A',14),('','A',6), ('','A',49)]
combs2.design.functions.write_resfile(template,
                                      CGs=['bb_cco', 'conh2'],
                                      outpath=outpath_resfile,
                                      filename='resfile_rotamer',
                                      tag='',
                                      resindices=None,
                                      segs_chains_resnums=segs_chains_resnums,
                                      pikaa_dict=None,
                                      bb_dep=1,
                                      use_enriched_vdMs=True,
                                      CA_burial_distance=None,
                                      exclude_exposed=False,
                                      exclude_intermed=False,
                                      exclude_buried=False,
                                      top_exposed=2,
                                      top_intermed=None,
                                      top_buried=None,
                                      alpha_hull_radius=9,
                                      use_propensities=True,
                                      propensity_threshold=0.9,
                                      use_abple=True,
                                      use_dssp=False,
                                      path_to_pdb_for_dssp=None,
                                      allowed_exposed='KRDENQSTMAGP',
                                      allowed_intermed='NQSTCMAGPVIL',
                                      allowed_buried='AGSTMCPVILHFWY',
                                      hb_only_residues='',
                                      all_contact_residues='HQY',
                                      rotamers=segs_chains_resnums)

In [35]:
# path to vdM databases
path_to_database='/Volumes/disk1/Combs2/database/20211005/vdMs/'

s = combs2.design._sample.Sample(**dict(path_to_resfile=input_dir + 'resfile_rotamer.txt',
                                        path_to_database=path_to_database))
s.read_resfile()
s.load_vdms(template, filter_by_phi_psi=False, run_parallel=False)

Loading bb_cco
    Loading GLN
        Loading residue ('', 'A', 14)
            Added  46 vdMs of 105 possible before clash filter.
    Loading TYR
        Loading residue ('', 'A', 6)
            Added  39 vdMs of 3713 possible before clash filter.
    Loading HIS
        Loading residue ('', 'A', 49)
            Added  5 vdMs of 449 possible before clash filter.
Loaded  90 vdMs of bb_cco
Loading conh2
    Loading GLN
        Loading residue ('', 'A', 14)
            Added  42 vdMs of 57 possible before clash filter.
    Loading TYR
        Loading residue ('', 'A', 6)
            Added  26 vdMs of 2258 possible before clash filter.
    Loading HIS
        Loading residue ('', 'A', 49)
            no vdms due to clashing CGs
Loaded  68 vdMs of conh2


In [36]:
path_to_ligand_file = input_dir + 'ligand.txt'
s.set_ligand_vdm_correspondence(path_to_ligand_file)
s.set_cg_neighbors(cg_max_dists=dict(conh2=0.8, bb_cco=0.8))

Setting neighbors...


Cluster the vdMs and print the top 10 clusters

In [37]:
output_dir = input_dir + 'output/'

s.cluster_vdms()
for i in range(1, 11):
    s.print_vdm_cluster(cg='conh2', cluster_number=i, outpath=output_dir + 'conh2_clusters/', tag='_clu' + str(i), prefix='')
    s.print_vdm_cluster(cg='bb_cco', cluster_number=i, outpath=output_dir + 'bb_cco_clusters/', tag='_clu' + str(i), prefix='')